# Feature Extraction using the PyRadiomics toolbox in Python

### Imports

In [1]:
import os  # needed navigate the system to get the input data
import pandas as pd
from pathlib import Path


import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Load the label data for each sample

In [4]:
# Load the clinical_data
dataset_folder = Path(os.getcwd()+'/dataset')
clinical_data = pd.read_excel(dataset_folder/Path('Pretreat-MetsToBrain-Masks_clin_20230918.xlsx'),sheet_name='Data')

# Select and process the clinical_data
# label = label[['BraTS_MET_ID', 'Death']].dropna(axis=0, subset=['Death'])
clinical_data = clinical_data.dropna(axis=0, subset=['Death'])
clinical_data.Death = clinical_data.Death.astype(int)

In [5]:
clinical_data

,BraTS_MET_ID,Age,Sex,Ethnicity,Smoke,Primary,Extranodal,Survival,Death,Infratentorial,ET_num,Nec_num,Edema_num,ET_vol,Nec_vol,Edema_vol,Nec_ET_ratio,Edema_ET_ratio
0,86,43.684932,1,0,2.5,1,1.0,92.166667,0,1,3,0,3,0.458,0.000,3.469,0.000000,7.574236
1,89,56.958904,1,0,9.0,4,1.0,21.266667,1,0,1,0,1,0.487,0.000,0.267,0.000000,0.548255
2,90,57.482192,1,0,37.5,6,0.0,27.000000,1,1,2,0,1,0.061,0.000,0.142,0.000000,2.327869
3,96,75.536986,0,0,20.0,5,0.0,84.166667,0,1,2,1,1,0.288,0.045,0.114,0.135135,0.342342
4,97,64.728767,1,0,NaN,5,0.0,4.666667,0,1,3,2,2,2.242,2.830,4.912,0.557965,0.968454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,305,53.460274,1,0,10.0,5,1.0,17.566667,1,1,6,2,5,2.283,0.284,2.252,0.110635,0.877289
196,306,69.515068,0,0,0.0,4,0.0,77.733333,0,0,7,1,6,29.150,0.807,145.732,0.026939,4.864706
197,307,53.624658,1,0,17.5,3,0.0,29.966667,0,0,1,1,1,1.450,0.721,81.500,0.332105,37.540304
198,309,84.709589,1,0,40.0,5,1.0,38.433333,1,0,2,0,2,0.130,0.000,0.168,0.000000,1.292308


### Instantiating the extractor

##### Important step to define the parms file version

In [6]:
# params_file_name = 'default_params'
params_file_name = 'params_2'

image_types = ('original', 'log-sigma', 'wavelet')

print(f"Setup for parameters: {params_file_name} and image types: {image_types}")

Setup for parameters: params_2 and image types: ('original', 'log-sigma', 'wavelet')


In [7]:
# Location of the parameter file
paramPath = os.path.join('params', params_file_name+'.yaml')
print('Parameter file, absolute path:', os.path.abspath(paramPath))

# Instantiate the extractor with parameters file
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Parameter file, absolute path: /Users/stelios/workspace/stelios/metstobrain-project/v2/params/params_2.yaml
Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': None, 'resampledPixelSpacing': [3, 2, 2], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 4, 'voxelArrayShift': 300}
Enabled filters:
	 {'Original': {}, 'LoG': {'sigma': [2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
Enabled features:
	 {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrl

### Setting up data and Feature Extraction

In [11]:
rootdir = dataset_folder/Path("dataset_to_process") # directory to store data we are interested in
features = [] # List of extracted features

for i, x in enumerate(os.walk(rootdir)):
    print(f"Sample: {i}")
    if 'BraTS' in x[0]:
        # sample_dir = /path/to/repo/v2/dataset/dataset_to_process/BraTS-MET-XXXXX-YYY
        sample_dir = x[0]

        # sample_file_base_name = BraTS-MET-XXXXX-YYY
        sample_file_base_name = sample_dir.split('/')[-1]

        # BraTS_MET_ID = XXXXX : int
        BraTS_MET_ID = int(sample_file_base_name.split('-')[-2])
        print(f"Fetching sample with ID: {BraTS_MET_ID}")

        # Get Image paths
        imagePath = os.path.join(sample_dir, sample_file_base_name+'-t2w.nii')
        maskPath = os.path.join(sample_dir, sample_file_base_name+'-seg.nii')

        # Feature extraction with pyradiomics
        print(f"Extracting features for sample: {sample_file_base_name}")
        try:
            results = extractor.execute(imagePath, maskPath)

            # Keep only specific features 
            temp = {k:v for k, v in results.items() if k.startswith(image_types)}
            temp.update({'BraTS_MET_ID':BraTS_MET_ID})
            features.append(temp)
        except Exception as e:
            print(f"Exception {e} for sample {imagePath} - {maskPath}")

Sample: 0
Sample: 1
Fetching sample with ID: 86
Extracting features for sample: BraTS-MET-00086-000
Sample: 2
Fetching sample with ID: 284
Extracting features for sample: BraTS-MET-00284-000
Sample: 3
Fetching sample with ID: 290
Extracting features for sample: BraTS-MET-00290-000
Sample: 4
Fetching sample with ID: 247
Extracting features for sample: BraTS-MET-00247-000
Sample: 5
Fetching sample with ID: 253
Extracting features for sample: BraTS-MET-00253-000
Sample: 6
Fetching sample with ID: 119
Extracting features for sample: BraTS-MET-00119-000
Sample: 7
Fetching sample with ID: 131
Extracting features for sample: BraTS-MET-00131-000
Sample: 8
Fetching sample with ID: 125
Extracting features for sample: BraTS-MET-00125-000
Sample: 9
Fetching sample with ID: 124
Extracting features for sample: BraTS-MET-00124-000
Sample: 10
Fetching sample with ID: 130
Extracting features for sample: BraTS-MET-00130-000
Sample: 11
Fetching sample with ID: 118
Extracting features for sample: BraTS-ME

### Store to Dataframe and merge with label data

In [13]:
len(features)

199

In [12]:
df = pd.DataFrame(features)
df = df.astype(float)
# df = pd.merge(df, clinical_data, how='left', on='BraTS_MET_ID')
# feature_output_filename = dataset_folder/Path('features')/Path(params_file_name)
feature_output_filename = os.path.join(dataset_folder, "features", 'just_features.pkl')
pd.to_pickle(df, feature_output_filename)
print(f"Stored dataset {feature_output_filename} to pickle")

Stored dataset /Users/stelios/workspace/stelios/metstobrain-project/v2/dataset/features/just_features.pkl to pickle
